In [1]:
import pandas as pd

In [5]:
excel = pd.ExcelFile("https://docs.google.com/spreadsheets/d/17o2TJJ3_pmrFsFNIdhxyPW3PFO0zyksoSVbrWzrDJoU/export?format=xlsx")
print(excel.sheet_names)  # See available sheet names
df = excel.parse(excel.sheet_names[0])  # Load the first sheet, or specify by name


['group1', 'group 2', 'group_3', 'group 4', 'group_5', 'Sheet5']


In [6]:
df.head()


,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock
0,NaN,Dark Blue,True,Brick,Rectangle,2 x 4,8,False,NaN,1
1,NaN,Green,True,Brick,Rectangle,2 x 4,8,False,NaN,1
2,NaN,Coral,True,Brick,Square,2 x 2,4,False,NaN,1
3,NaN,Red,True,Brick,Square,2 x 2,4,False,NaN,1
4,NaN,White,True,Brick,Square,2 x 2,4,False,NaN,1


In [7]:
# Read all sheets into a list of DataFrames
dfs = [excel.parse(sheet_name) for sheet_name in excel.sheet_names]

# Combine them all together (stack one below the other)
df = pd.concat(dfs, ignore_index=True)


In [8]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               0 non-null      object 
 1   color            204 non-null    object 
 2   is duplo?        204 non-null    object 
 3   size type        204 non-null    object 
 4   base shape       204 non-null    object 
 5   base dimensions  204 non-null    object 
 6   number of studs  204 non-null    float64
 7   has slope?       204 non-null    object 
 8   slope degree     168 non-null    float64
 9   in stock         204 non-null    object 
 10  transparent      40 non-null     float64
dtypes: float64(3), object(8)
memory usage: 17.7+ KB


,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock,transparent
0,NaN,Dark Blue,True,Brick,Rectangle,2 x 4,8.0,False,NaN,1,NaN
1,NaN,Green,True,Brick,Rectangle,2 x 4,8.0,False,NaN,1,NaN
2,NaN,Coral,True,Brick,Square,2 x 2,4.0,False,NaN,1,NaN
3,NaN,Red,True,Brick,Square,2 x 2,4.0,False,NaN,1,NaN
4,NaN,White,True,Brick,Square,2 x 2,4.0,False,NaN,1,NaN


In [9]:
df.tail(5)

,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock,transparent
199,NaN,Transparent orange,False,Plate,Square,1*1,0.0,False,0.0,1,NaN
200,NaN,Blck,False,Plate,Trapezium,2*2,4.0,False,0.0,1,NaN
201,NaN,Yellow,False,Plate,Rectangle,1*2,0.0,False,0.0,1,NaN
202,NaN,Transparent sky blue,False,Plate,Round,1*1,0.0,False,0.0,1,NaN
203,NaN,Transparent yellow,False,Plate,Round,1*1,0.0,False,0.0,1,NaN
